## Data Collection Class

In [22]:
class DataCollection:
    _id = ""
    name = ""
    provider = ""
    licence = ""
    size = 0
    author = ""
    description = ""        
    
    releaseList = []
 
    def __init__(self, _id, name, provider, licence, size, author, description, releaseList):
        self._id = _id
        self.name = name
        self.provider = provider
        self.licence = licence
        self.size = size
        self.author = author
        self.description = "Add API decription"
        
        self.releaseList = releaseList
 
    def printInfo(self):
        print ("_id  = " + self._id)
        print ("name  = " + self.name)
        print ("provider  = " + self.provider)
        print ("licence  = " + self.licence)
        print ("size = " + str(self.size) + " Bytes")
        print ("author  = " + self.author)
        print ("description  = " + self.description)
        print ("list of releases = " + str(self.releaseList))      
        
print ("Success class created!")

Success class created!


## Release Class

In [23]:
# Get conditions in which data is produced
class Release:
    _id = ""
    releaseNum = 0
    publicationDate = ""    
    
    itemList = []
    size = 0   # added by me
    
    def __init__(self, _id, releaseNum, publicationDate, itemList, size):
        self._id = _id
        self.releaseNum = releaseNum
        self.publicationDate = publicationDate        
        self.itemList = itemList
        self.size = size
        
    def incRelease(self):
        self.releaseNum = self.releaseNum + 1    
        
    def printInfo(self):
        print ("_id  = " + self._id)
        print ("releaseNum  = " + str(self.releaseNum))
        print ("publicationDate  = " + self.publicationDate)                
        print ("List of items = " + str(self.itemList))
        print ("size = " + str(self.size) + " Bytes")

print ("Success class created!")

Success class created!


## Item Class

In [11]:
class Item:
    _id = ""
    name = ""
    content = []
    
    size = 0   # added by me
 
    def __init__(self, _id, name, content, size):
        self._id = _id
        self.name = name
        self.content = content
        self.size = size
        
    def printInfo(self):
        print ("_id  = " + self._id)
        print ("name  = " + self.name)
        print ("content  = " + str(self.content))
        print ("size = " + str(self.size) + " Bytes")

print ("Success class created!")

Success class created!


## Harvest Data from HTTP and Unarchived it

In [5]:
import urllib.request
import shutil
from os import listdir
import zipfile

url = "http://ergast.com/downloads/f1db_csv.zip"
tempPath = '/tmp/'
file_name = tempPath + "f1db_csv.zip"
dataFolder = "f1db_csv/"

# Download the file from 'url' and save it locally under 'file_name':
with urllib.request.urlopen(url) as response, open(file_name, 'wb') as out_file:
    shutil.copyfileobj(response, out_file)        

# Unarchive data
zip_ref = zipfile.ZipFile(file_name, 'r')
zip_ref.extractall(tempPath + dataFolder) # creates folder f1db_csv if it doesn't exist
zip_ref.close()

listFilePath = os.listdir(tempPath + dataFolder)

print ("Data set has been downloaded, the following files were extracted extracted:")

for f in listFilePath:  
    fullPath = tempPath + dataFolder + f
    print (fullPath)

Data set has been downloaded, the following files were extracted extracted:
/tmp/f1db_csv/pit_stops.csv
/tmp/f1db_csv/races.csv
/tmp/f1db_csv/lap_times.csv
/tmp/f1db_csv/constructors.csv
/tmp/f1db_csv/constructor_results.csv
/tmp/f1db_csv/status.csv
/tmp/f1db_csv/driver.csv
/tmp/f1db_csv/seasons.csv
/tmp/f1db_csv/driver_standings.csv
/tmp/f1db_csv/qualifying.csv
/tmp/f1db_csv/constructor_standings.csv
/tmp/f1db_csv/circuits.csv
/tmp/f1db_csv/results.csv


### Create item objects and append them to a list

In [12]:
# Get list of items <br />
# Get Items' content, name and size <br />
# Create a list of item objects

import os 
from pyspark import SparkContext

## ITEM objects ###
itemList = []
index = 0

# Ask to introduce schema info if known, else infere schema

for f in listFilePath:  
    fullPath = tempPath + dataFolder + f
    df = sqlContext.read.format('com.databricks.spark.csv').options(header='false', inferschema='true').load(fullPath) # note: custom schema should be defined in deduction phase (later)
    content = df.dtypes # list N-tuple <attribute, data type>
    size = os.path.getsize(fullPath) # get file size in Bytes
    ## Instance
    itemList.append(Item(url, f, content, size)) # _id, name, content, size
    itemList[index].printInfo()
    print() # Add a blank space between instances
    index+=1

_id  = http://ergast.com/downloads/f1db_csv.zip
name  = pit_stops.csv
content  = [('_c0', 'int'), ('_c1', 'int'), ('_c2', 'int'), ('_c3', 'int'), ('_c4', 'string'), ('_c5', 'string'), ('_c6', 'int')]
size = 254006 Bytes

_id  = http://ergast.com/downloads/f1db_csv.zip
name  = races.csv
content  = [('_c0', 'int'), ('_c1', 'int'), ('_c2', 'int'), ('_c3', 'int'), ('_c4', 'string'), ('_c5', 'timestamp'), ('_c6', 'string'), ('_c7', 'string')]
size = 109577 Bytes

_id  = http://ergast.com/downloads/f1db_csv.zip
name  = lap_times.csv
content  = [('_c0', 'int'), ('_c1', 'int'), ('_c2', 'int'), ('_c3', 'int'), ('_c4', 'string'), ('_c5', 'int')]
size = 12993131 Bytes

_id  = http://ergast.com/downloads/f1db_csv.zip
name  = constructors.csv
content  = [('_c0', 'int'), ('_c1', 'string'), ('_c2', 'string'), ('_c3', 'string'), ('_c4', 'string')]
size = 17021 Bytes

_id  = http://ergast.com/downloads/f1db_csv.zip
name  = constructor_results.csv
content  = [('_c0', 'int'), ('_c1', 'int'), ('_c2', 'int

### Create release objects and append them in a list 

In [24]:
############# RELEASE ###############################################################################################
# We assume that publication date is the same as the one which the origin server believes the resource was last modified
conn = urllib.request.urlopen("http://ergast.com/downloads/f1db_csv.zip", timeout=30)
last_modified = conn.headers['last-modified']
publicationDate = last_modified

# Get release size <br />
releaseSize = 0
for i in itemList:
    releaseSize = i.size + releaseSize

# Create a list of release objects    
releaseList = []
                           #_id, releaseNum, publicationDate, itemList, size
releaseList.append(Release(url, 1, publicationDate, itemList, releaseSize)) 
releaseList[0].printInfo()
#####################################################################################################################

_id  = http://ergast.com/downloads/f1db_csv.zip
releaseNum  = 1
publicationDate  = Sun, 02 Sep 2018 18:08:37 GMT
List of items = [<__main__.Item object at 0x7f362c3b9f98>, <__main__.Item object at 0x7f362c3b9048>, <__main__.Item object at 0x7f362c3b9630>, <__main__.Item object at 0x7f362c3be080>, <__main__.Item object at 0x7f362c3bee48>, <__main__.Item object at 0x7f362c3b51d0>, <__main__.Item object at 0x7f362c3b5b00>, <__main__.Item object at 0x7f362c396f98>, <__main__.Item object at 0x7f362c3b1588>, <__main__.Item object at 0x7f362c3b1c88>, <__main__.Item object at 0x7f362c14cb00>, <__main__.Item object at 0x7f362c14c4e0>, <__main__.Item object at 0x7f362c3b5630>]
size = 16605039 Bytes


### Create data collection object
Using release size since both release and collection have the same data <br />
Adding parameter manually, consider extracting them programmatically

In [25]:
############# COLLECTION ############################################################################################
# Ask for database name, provider, licence, author and description
# _id = url maybe change for a hdfs url
                                #_id, name, provider, licence, size, author, description, releaseList
dataCollection = DataCollection(url, dataFolder, "Ergast Developer API", "Restricted", releaseSize, "ergast", "description", releaseList) 
dataCollection.printInfo()
#####################################################################################################################

_id  = http://ergast.com/downloads/f1db_csv.zip
name  = f1db_csv/
provider  = Ergast Developer API
licence  = Restricted
size = 16605039 Bytes
author  = ergast
description  = Add API decription
list of releases = [<__main__.Release object at 0x7f362c14c358>]



### GUI

In [2]:
import ipywidgets as widgets